In [1]:
import matplotlib.pyplot as plt
import numpy as np
import dask.array as da
import zarr as zr
import django
from dask.distributed import Client, progress

django.setup()
from bioconverter.models import Representation

Debugging build
Hallo


In [2]:
c = Client()
c

Client Scheduler: tcp://daskscheduler:8786 Dashboard: http://daskscheduler:8787/status,Cluster Workers: 1 Cores: 4 Memory: 4.14 GB


In [3]:
it = Representation.distributed.filter(zarr__isnull=False)

In [4]:
it.first().loadArray().shape

(1024, 1024, 3, 21, 1)

In [5]:
arrays = it.asArrays().map(lambda x: x.sel(time=0)).filter(lambda x: len(x.shape) == 4).filter(lambda item: item.shape[3] == 21)

In [6]:
arrays.compute()

[<xarray.DataArray 'data' (x: 1024, y: 1024, channels: 3, z: 21)>
 dask.array<getitem, shape=(1024, 1024, 3, 21), dtype=float16, chunksize=(256, 256, 1, 11), chunktype=numpy.ndarray>
 Coordinates:
   * channels  (channels) <U1 'R' 'G' 'B'
     time      int64 0
   * x         (x) float64 0.0 0.65 1.3 1.95 2.6 ... 663.0 663.6 664.3 665.0
   * y         (y) float64 0.0 0.65 1.3 1.95 2.6 ... 663.0 663.6 664.3 665.0
   * z         (z) float64 0.0 0.65 1.3 1.95 2.6 ... 10.4 11.05 11.7 12.35 13.0
 Attributes:
     units:    µm,
 <xarray.DataArray 'data' (x: 1024, y: 1024, channels: 3, z: 21)>
 dask.array<getitem, shape=(1024, 1024, 3, 21), dtype=float16, chunksize=(256, 256, 1, 11), chunktype=numpy.ndarray>
 Coordinates:
   * channels  (channels) <U1 'R' 'G' 'B'
     time      int64 0
   * x         (x) float64 0.0 0.65 1.3 1.95 2.6 ... 663.0 663.6 664.3 665.0
   * y         (y) float64 0.0 0.65 1.3 1.95 2.6 ... 663.0 663.6 664.3 665.0
   * z         (z) float64 0.0 0.65 1.3 1.95 2.6 ... 10.

In [7]:
import dask.array as da
s = da.stack(arrays, axis=0)

In [8]:
s

dask.array<stack, shape=(5, 1024, 1024, 3, 21), dtype=float16, chunksize=(1, 256, 256, 1, 11), chunktype=numpy.ndarray>

In [13]:
new = s.persist()

In [16]:
new = new * 2

In [17]:
new.persist()

dask.array<mul, shape=(5, 1024, 1024, 3, 21), dtype=float16, chunksize=(1, 256, 256, 1, 11), chunktype=numpy.ndarray>